# Import dependencies

In [127]:
# from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from sqlalchemy import create_engine

# Datasets

In [155]:
unemployment_csv_file = "input/unemployment_rate.csv"
unemployment_df = pd.read_csv(unemployment_csv_file, encoding ='cp1252')
unemployment_df["State"] = unemployment_df["State"].astype(str)
unemployment_df.head()

,Series ID,Region Name,State,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,SCABBE1URN,Abbeville County,SC,3.8,6.6,7.7,9.1,8.2,7.7,8.8,...,14.2,13.6,12.5,10.6,9.5,7.7,6.9,5.5,4.6,4.2
1,LAACAD0URN,Acadia Parish,LA,5.8,6.0,6.3,6.8,5.8,6.1,3.6,...,6.5,7.3,6.7,6.0,6.0,5.8,6.9,7.4,5.9,5.4
2,VAACCO1URN,Accomack County,VA,3.0,3.4,4.2,4.4,4.7,4.7,4.3,...,6.5,7.9,8.4,7.9,7.2,6.7,5.5,5.0,4.9,4.1
3,IDADAC1URN,Ada County,ID,3.4,3.9,4.7,4.9,4.1,3.3,2.7,...,8.9,8.3,7.3,6.2,5.2,4.2,3.7,3.3,2.8,2.4
4,OKADAI1URN,Adair County,OK,3.5,4.1,4.8,7.8,6.2,4.8,4.4,...,8.0,10.5,9.2,9.1,9.8,8.0,6.1,5.9,5.0,4.4


In [156]:
income_csv_file ="input/income_data.csv"
income_df = pd.read_csv(income_csv_file, encoding = "UTF-8")
income_df["State_ab"] = income_df["State_ab"].astype(str)
income_df.head()

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


In [157]:
auto_ins_csv_file = "input/auto_insurance_cost.csv"
auto_ins_data_df = pd.read_csv(auto_ins_csv_file)
auto_ins_data_df.head()

,rank,state,full_coverage,minimum_coverage,difference
0,1,Michigan,8723,5282,3441
1,2,Rhode Island,3847,1589,2258
2,3,Louisiana,3525,1329,2196
3,4,Kentucky,3418,1338,2079
4,5,Florida,3370,2565,805


# Create dataframe

In [158]:
unemployment_df = unemployment_df[["State", "2018" ]]
clean_unemployment = round(unemployment_df.groupby("State").mean(),2)
clean_unemployment = clean_unemployment.reset_index()
clean_unemployment.columns = ["State", "Unemployment Rate"]
clean_unemployment

,State,Unemployment Rate
0,AK,8.53
1,AL,4.53
2,AR,4.24
3,AZ,6.65
4,CA,5.19
5,CO,3.17
6,CT,4.10
7,DC,5.60
8,DE,4.03
9,FL,3.75


In [159]:
income_df = income_df[["State_ab", "Mean" ]]
clean_income = round(income_df.groupby("State_ab").mean(),2)
clean_income = clean_income.reset_index()
clean_income.columns = ["State", "Income"]
clean_income

,State,Income
0,AK,77670.21
1,AL,53612.93
2,AR,52060.36
3,AZ,62578.07
4,CA,78126.74
5,CO,73322.82
6,CT,89227.22
7,DC,90668.42
8,DE,70142.34
9,FL,60887.89


In [160]:
clean_unemployment["State"] = clean_unemployment["State"].str.strip()
clean_unemployment

,State,Unemployment Rate
0,AK,8.53
1,AL,4.53
2,AR,4.24
3,AZ,6.65
4,CA,5.19
5,CO,3.17
6,CT,4.10
7,DC,5.60
8,DE,4.03
9,FL,3.75


In [161]:
clean_income["State"] = clean_income["State"].str.strip()
clean_income

,State,Income
0,AK,77670.21
1,AL,53612.93
2,AR,52060.36
3,AZ,62578.07
4,CA,78126.74
5,CO,73322.82
6,CT,89227.22
7,DC,90668.42
8,DE,70142.34
9,FL,60887.89


In [165]:
merge_df = clean_unemployment.merge(clean_income, how = "left", on = "State")
merge_df

,State,Unemployment Rate,Income
0,AK,8.53,77670.21
1,AL,4.53,53612.93
2,AR,4.24,52060.36
3,AZ,6.65,62578.07
4,CA,5.19,78126.74
5,CO,3.17,73322.82
6,CT,4.10,89227.22
7,DC,5.60,90668.42
8,DE,4.03,70142.34
9,FL,3.75,60887.89
